# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [76]:
import warnings
warnings.filterwarnings("ignore")

In [126]:
def F(a, b, n):
    return float(b - a**2)/n

def get_threshold(feature, y):
    best_thr = None
    data = np.hstack((feature.reshape(-1, 1), y.reshape(-1, 1)))
    data = data[data[:, 0].argsort()]
    best_thr = data[0, 0]
    n = data.shape[0]
                     
    left = data[data[:, 0] < best_thr]
                     
    left_sum = np.sum(left[:, 1])
    left_sqr = left[:, 1] @ left[:, 1]
    l_n = left.shape[0]
                     
    right = data[data[:, 0] >= best_thr]
                     
    right_sum = np.sum(right[:, 1])
    right_sqr = right[:, 1] @ right[:, 1]
    r_n = right.shape[0]
    
    best_F = float('inf')
     
    for i in range(n-1):
        obj = data[i, 0]
        target = data[i, 1]
        l_n += 1
        r_n -= 1
        left_sum += target
        left_sqr += target**2
        right_sum -= target
        right_sqr -= target**2
        F_l = F(left_sum, left_sqr, l_n)
        F_r = F(right_sum, right_sqr, r_n)

        if F_l + F_r < best_F:
            best_thr = obj
            left_mean = float(left_sum)/l_n
            right_mean = float(right_sum)/r_n
            best_F = F_l + F_r
    return best_F, best_thr, left_mean, right_mean


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = float('inf')
    left_mean, right_mean = None, None
    for j in range(X.shape[1]):
        f_score, f_thr, l_mean, r_mean = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_error = f_score
            best_f = j
            best_thr = f_thr
            left_mean = l_mean
            right_mean = r_mean
    return best_f, best_thr, left_mean, right_mean

def my_pred(X, best_feat, best_thr, left_mean, right_mean):
    y = np.where(X[:, best_feat] < best_thr, left_mean, right_mean)
    return y
    

In [127]:
from sklearn.datasets import load_boston
import numpy as np
X, y = load_boston(return_X_y=True)

In [128]:
best_feat, best_thr, left_mean, right_mean = train_stump(X, y)

In [129]:
best_feat, best_thr, left_mean, right_mean

(12, 9.71, 29.729245283018848, 17.34353741496599)

In [130]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

print(mean_squared_error(y, my_pred(X, best_feat, best_thr, left_mean, right_mean)))


0.0
46.19909167710848
47.23017483291336
